In [25]:
!pip list

Package                  Version              
------------------------ ---------------------
absl-py                  0.7.1                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.2.0                
astor                    0.8.0                
astroid                  2.2.5                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.1.0               
audioread                2.1.8                
autograd                 1.3                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   1.5.0                
blis                     0.2.4                
bokeh        

In [0]:
import numpy as np
import pandas as pd
import os
import random
from collections import deque
from IPython.display import clear_output

clear_output()

DATADIR = ""

SEQUENCE_LEN = 50  # Number of previous data points to use in RNN
FUTURE_LEN = 5  # Number of future data points to predict

In [9]:
# Reading the data
# ---------------------------------------------------------

dataset = pd.read_csv( os.path.join(DATADIR, "ETH-USD.csv"), names=['time', 'low', 'high', 'open', 'close', 'volume'] )
dataset.fillna(method="ffill", inplace=True)  # if there are gaps in data, use previously known values
dataset.dropna(inplace=True)
dataset.set_index("time", inplace=True)

dataset['future'] = dataset['close'].shift(-FUTURE_LEN)


def increaseOrDecrease(past, future):
    # Figure out whether the value increses or decreases
    if float(past) < float(future):
        return 1  # increase
    else:
        return 0  # decrease or static


dataset['trend'] = list(map(increaseOrDecrease, dataset['close'], dataset['future']))

val_split = 0.05
val_split_indices = sorted(dataset.index.values)[-int( val_split * len(sorted(dataset.index.values)) )]
validation_dataset = dataset[(dataset.index >= val_split_indices )]

discard_indices = sorted(dataset.index.values)[-int( 5 )]  # Discard NaN values due to shift
validation_dataset = validation_dataset[(validation_dataset.index <= discard_indices)]

dataset = dataset[(dataset.index < val_split_indices)]

print(dataset.head())

# Drop unnecessary columns
validation_dataset = validation_dataset.drop("low", 1)
validation_dataset = validation_dataset.drop("high", 1)
validation_dataset = validation_dataset.drop("open", 1)
validation_dataset = validation_dataset.drop("future", 1)

dataset = dataset.drop("low", 1)
dataset = dataset.drop("high", 1)
dataset = dataset.drop("open", 1)
dataset = dataset.drop("future", 1)

                  low   high        open  ...     volume     future  trend
time                                      ...                             
1528968720  485.98999  486.5  486.019989  ...  26.019083  485.98999      0
1528968780  486.00000  486.0  486.000000  ...   8.449400  485.98999      0
1528968840  485.75000  486.0  486.000000  ...  26.994646  485.98999      1
1528968900  485.75000  486.0  485.750000  ...  77.355759  485.98999      0
1528968960  485.98999  486.0  486.000000  ...   7.503300  486.00000      0

[5 rows x 7 columns]


In [0]:

# Data Preprocessing
# ---------------------------------------------------------

from sklearn import preprocessing

def preprocess(df):
    for col in df.columns:
        if col != "trend":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
        
    df.dropna(inplace=True)

    # Contains all sequences of required size
    sequences = []
    # Current sequence, deque pushes out old data as new entries are appended
    seq = deque(maxlen=SEQUENCE_LEN)

    for i in df.values:
        seq.append([n for n in i[:-1]])  # Append everything except last column
        if len(seq) == SEQUENCE_LEN:
            sequences.append([np.array(seq), i[-1]])
            # Add the current sequence
    
    # Shuffle order of sequences
    random.shuffle(sequences)

    # Splitting into buys and sells
    buys = []
    sells = []

    for _seq, trend in sequences:
        if trend == 0:
            buys.append([_seq, trend])
        elif trend == 1:
            sells.append([_seq, trend])

    random.shuffle(buys)
    random.shuffle(sells)

    # Equalize the two
    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]

    # Rejoin into main
    sequences = buys + sells
    random.shuffle(sequences)

    # FINALLY Split into features and labels
    X = []
    y = []

    for _seq, trend in sequences:
        X.append(_seq)
        y.append(trend)

    X = np.array(X)

    return X, y


train_X, train_y = preprocess(dataset)
validation_X, validation_y = preprocess(validation_dataset)

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [12]:
# Creating the Model
# ---------------------------------------------------------

# Sequential Model
model = Sequential()

# Layer 1 :: Sheets = LSTM, Dropout, Normalize
# Input layer from the sequences
model.add(CuDNNLSTM(128, input_shape=(train_X.shape[1:]), return_sequences=True ))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Layer 2 :: Sheets = LSTM, Dropout, Normalize
model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Layer 3 :: Sheets = LSTM, Dropout, Normalize
# Final layer
model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Layer 4 :: Sheets = Dense, Dropout
# Collection layer, from 128 to 32 nodes
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(2, activation="softmax"))

LEARNING_RATE = 0.001
LEARNING_DECAY = 1e-6

EPOCHS = 10
BATCH_SIZE = 64

opt = Adam(lr=LEARNING_RATE, decay=LEARNING_DECAY)

W0830 17:13:46.537683 140074459232128 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Train Model
model.fit(
    train_X, train_y,
    batch_size=BATCH_SIZE, epochs=EPOCHS,
    validation_data=(validation_X, validation_y)
)